In [ ]:
from tkinter import Tk, filedialog
from google.colab import drive
from __future__ import absolute_import, print_function, division, unicode_literals
import os
import glob
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, Dense
from keras import regularizers

# Function to open the file dialog and get the selected directory
def select_dataset_directory():
    root = Tk()
    root.withdraw()  # Hide the main window
    dataset_dir = filedialog.askdirectory(title="Select Dataset Directory")
    root.destroy()
    return dataset_dir

# Mount Google Drive
drive.mount('/content/drive')

# Select the dataset directory using the GUI
data_dir = select_dataset_directory()
data_dir = pathlib.Path(data_dir)

BATCH_SIZE = 8
IMG_HEIGHT = 224
IMG_WIDTH = 224

CLASS_NAMES = np.array(['Bacterial leaf blight', 'Brown Spot', 'Leaf Smut'])

print('Class Names: ', CLASS_NAMES)

train_path = '/content/drive/MyDrive/Dataset'
test_path = '/content/drive/MyDrive/Dataset'

image_train_gen = ImageDataGenerator(rescale=1. / 255,
                                     zoom_range=0.50,
                                     rotation_range=45,
                                     horizontal_flip=True,
                                     width_shift_range=0.15,
                                     height_shift_range=0.15)

train_data_gen = image_train_gen.flow_from_directory(train_path,
                                                     shuffle=True,
                                                     batch_size=BATCH_SIZE,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='sparse')

img_val_gen = ImageDataGenerator(rescale=1. / 255)
val_data_gen = img_val_gen.flow_from_directory(test_path,
                                               batch_size=BATCH_SIZE,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH),
                                               class_mode='sparse')

def plotImages(image_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(image_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

# Plot a few training images
img_array = [train_data_gen[0][0][0] for i in range(5)]
plotImages(img_array)

# Plot a few validation images
img_array = [val_data_gen[0][0][0] for i in range(5)]
plotImages(img_array)

# Model building
# Instantiate a ConvNet
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(224, 224, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(3, activation="softmax"))

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

model.summary()

EPOCHS = 70
history = model.fit_generator(train_data_gen, epochs=EPOCHS, validation_data=val_data_gen)

# Plot training and validation graphs
acc = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()